In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.now())
print(f"{current_date}")

2025-05-03 16:09:50.066573


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date =current_date.floor("h")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-05-03T16:00:00.000000000')

In [10]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=32)

In [8]:
fetch_data_to

Timestamp('2025-05-03 16:00:00')

In [11]:
fetch_data_from

Timestamp('2025-04-01 16:00:00')

In [17]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_bike_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_bike_data(years=[historical_from_date.year], months=[historical_from_date.month])
    rides_from = rides_from[rides_from.started_at >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_bike_data(years=[historical_to_date.year], months=[historical_to_date.month])
        rides_to = rides_to[rides_to.started_at < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['started_at'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['start_station_id', 'started_at'], inplace=True)

    return rides

In [18]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-04.
Loading data for 2024-04...
Total records: 3,217,063
Valid records: 34,543
Records dropped: 3,182,520 (98.93%)
Successfully processed data for 2024-04.
Combining all monthly data...
Data loading and processing for 2024 complete!
Combining all data...
Data loading and processing complete!
File already exists for 2024-05.
Loading data for 2024-05...
Total records: 4,230,360
Valid records: 40,809
Records dropped: 4,189,551 (99.04%)
Successfully processed data for 2024-05.
Combining all monthly data...
Data loading and processing for 2024 complete!
Combining all data...
Data loading and processing complete!


In [19]:
rides

,started_at,start_station_id
11,2025-04-01 16:32:13.646,5905.14
13,2025-04-01 16:47:35.223,5905.14
22,2025-04-01 16:56:18.669,5905.14
27,2025-04-01 17:02:18.267,5905.14
29,2025-04-01 17:02:48.671,5905.14
...,...,...
38707,2025-05-03 15:50:51.000,6822.09
38714,2025-05-03 15:54:41.000,6822.09
38720,2025-05-03 15:56:54.000,6822.09
38721,2025-05-03 15:57:02.000,6822.09


In [20]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [21]:
ts_data

,start_hour,start_station_id,rides
0,2025-04-01 16:00:00,5905.140137,3
1,2025-04-01 17:00:00,5905.140137,5
2,2025-04-01 18:00:00,5905.140137,8
3,2025-04-01 19:00:00,5905.140137,5
4,2025-04-01 20:00:00,5905.140137,1
...,...,...,...
2299,2025-05-03 11:00:00,6822.089844,20
2300,2025-05-03 12:00:00,6822.089844,16
2301,2025-05-03 13:00:00,6822.089844,25
2302,2025-05-03 14:00:00,6822.089844,13


In [22]:
import pytz

est = pytz.timezone('US/Eastern')

ts_data["start_hour"] = ts_data["start_hour"].dt.tz_localize(est, ambiguous="NaT", nonexistent="NaT")
ts_data = ts_data.dropna(subset=["start_hour"])
ts_data

,start_hour,start_station_id,rides
0,2025-04-01 16:00:00-04:00,5905.140137,3
1,2025-04-01 17:00:00-04:00,5905.140137,5
2,2025-04-01 18:00:00-04:00,5905.140137,8
3,2025-04-01 19:00:00-04:00,5905.140137,5
4,2025-04-01 20:00:00-04:00,5905.140137,1
...,...,...,...
2299,2025-05-03 11:00:00-04:00,6822.089844,20
2300,2025-05-03 12:00:00-04:00,6822.089844,16
2301,2025-05-03 13:00:00-04:00,6822.089844,25
2302,2025-05-03 14:00:00-04:00,6822.089844,13


In [24]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2304 entries, 0 to 2303
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype                     
---  ------            --------------  -----                     
 0   start_hour        2304 non-null   datetime64[ns, US/Eastern]
 1   start_station_id  2304 non-null   float32                   
 2   rides             2304 non-null   int16                     
dtypes: datetime64[ns, US/Eastern](1), float32(1), int16(1)
memory usage: 31.6 KB


In [25]:
ts_data.tail()

,start_hour,start_station_id,rides
2299,2025-05-03 11:00:00-04:00,6822.089844,20
2300,2025-05-03 12:00:00-04:00,6822.089844,16
2301,2025-05-03 13:00:00-04:00,6822.089844,25
2302,2025-05-03 14:00:00-04:00,6822.089844,13
2303,2025-05-03 15:00:00-04:00,6822.089844,18


In [30]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')

# Initialize connection to Hopsworks
project = hopsworks.login(
    api_key_value=api_key,
    project=project_name
)
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-05-03 16:34:16,792 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-03 16:34:16,799 INFO: Initializing external client
2025-05-03 16:34:16,800 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 16:34:17,627 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Successfully connected to Hopsworks project: nolanphi


In [31]:
feature_store = project.get_feature_store()

In [32]:
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

In [33]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 2304/2304 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: bike_time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212635/jobs/named/bike_time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('bike_time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)